In [ ]:
:! stack build lazy-csv --silent
:! stack build ihaskell-charts --silent

putStrLn "Done! Please make sure to restart your kernel before proceeding (no need to run this cell again after)"

In [ ]:
-- Make sure to run this cell, otherwise your output will be very noisy.
:opt no-lint

# Functional Programming and Why It's Relevant for HEP: Exercises, Bonus

For this bonus exercise, your task is to implement the J/psi analysis from [Part 2](./Part2.ipynb) of this exercise session in Haskell.

First, let's define the data representation of our particle. We do this through a custom-made [data type](https://mmhaskell.com/liftoff/data-types) that will store the values the `pt`, `eta`, `phi` and `charge` of a particle. It would be useful to have some convenience functions to extract these values from a `Particle`. The one for getting the `pt` is given. As you see, this relies heavily on pattern matching!

Implement similar functions for getting the `eta`, `phi` and `charge`.

In [ ]:
type Pt = Float
type Eta = Float
type Phi = Float
type Charge = Int
data Particle = Particle Pt Eta Phi Charge
                deriving Show

pt :: Particle -> Float
pt (Particle p _ _ _) = p

eta :: Particle -> Float
eta (Particle _ e _ _) = e

phi :: Particle -> Float
phi (Particle _ _ p _) = p
                
charge :: Particle -> Int
charge (Particle _ _ _ c) = c

Next, we need some code to actually read our data. Here's where things get a little bit iffy. Reading from (and writing to) files is a side effect, which in principle is not allowed in Haskell! However, there is a way to still make (some) side effect possible. This is done through [monads](https://stackoverflow.com/questions/44965/what-is-a-monad). For now, you don't need to understand what exactly these are, just that there is the `IO` monad, which, depending on the function will need to be added to the function declaration. We will take care of this for you for this exercise, but if you're feeling brave, you can try to explore the mechanisms behind monads a bit!

In [ ]:
import Text.CSV.Lazy.String

events :: IO [(CSVRow, CSVRow)]
events = do
    content <- readFile "../data/df014_CsvDataSource_MuRun2010B.csv"
    let e = csvTable $ parseCSV content
    let m1 = case selectFields ["pt1", "eta1", "phi1", "Q1"] e of
                Left _    -> error "some fields not found"
                Right res -> drop 1 res
    let m2 = case selectFields ["pt2", "eta2", "phi2", "Q2"] e of
                Left _    -> error "some fields not found"
                Right res -> drop 1 res
    return $ zip m1 m2

muon :: CSVRow -> Particle
muon r = let [fPt, fEta, fPhi, fCharge] = map csvFieldContent r
          in Particle (rf fPt) (rf fEta) (rf fPhi) (ri fCharge)
          where rf = read :: String -> Float
                ri = read :: String -> Int

dimuon :: (CSVRow, CSVRow) -> (Particle, Particle)
dimuon (r1, r2) = (muon r1, muon r2)

dimuons :: IO [(Particle, Particle)]
dimuons = map dimuon <$> events

Let's see what our dimuon representation looks like, by printing the first element (the head!). Notice the `<$>` operator. This is required because `dimuons` is wrapped in this previously mentioned `IO` monad. Without going into too much detail, this operator will be required for any operation called on `dimuons` or anything that is derived from this list going forward.

In [ ]:
head <$> dimuons

As you can see, each element is a tuple containing two `Particle`s. Keep this in mind when defining your analysis functions!

Let's start by filtering only the dimuons with opposite charge. Write function `filter_dimuons` that does exactly that. Since this is a bonus challenge, we won't provide the type declarations anymore ;).

In [ ]:
filter_dimuons :: [(Particle, Particle)] -> [(Particle, Particle)]
filter_dimuons = filter (\ (p1, p2) -> (charge p1) * (charge p2) == -1)

After applying this filter to `dimuons`, 63946 events should remain. You can check this with the next cell:

In [ ]:
filtered_dimuons = filter_dimuons <$> dimuons
length <$> filtered_dimuons

Next up, it's time to calculate the invariant mass. Start by defining a function that will calculate this for just one pair of particles. The return type should be a `Float`.

In [ ]:
calc_invariant_mass :: (Particle, Particle) -> Float
calc_invariant_mass (p1, p2) = sqrt $ (2 * pt p1 * pt p2) * (cosh (eta p1 - eta p2) - cos (phi p1 - phi p2))

Of course, now we need to calculate the invariant mass for every event that survived the filter. Let's do this in the next cell:

In [ ]:
calc_invariant_masses :: [(Particle, Particle)] -> [Float]
calc_invariant_masses = map calc_invariant_mass

The invariant masses of the first three events should be equal to 3.0953546 9.4090395 and 8.674597. Use the `take` function to check whether this is the case:

In [ ]:
invariant_masses = calc_invariant_masses <$> filtered_dimuons

take 3 <$> invariant_masses

Now, let's apply the cut for the J/psi particle. We want to keep only the events where the invariant mass is within the range  $[2.95, 3.25]$. Write the function `apply_jpsi_cut` in the cell below:

In [ ]:
apply_jpsi_cut :: [Float] -> [Float]
apply_jpsi_cut = filter (\ p -> p > 2.95 && p < 3.25)

After this cut, 10027 events should remain. Verify this using the cell below:

In [ ]:
jpsis = apply_jpsi_cut <$> invariant_masses

length <$> jpsis

As the final step, of course we should plot the J/psi particle we found as a histogram! The function to produce such a histogram has already been given. The only thing left to do for you is call `plot_jpsi` with the results of your analysis!

In [ ]:
import IHaskell.Display
import Graphics.Rendering.Chart.Easy
import Graphics.Rendering.Chart.Plot.Histogram(defaultNormedPlotHist)


instance Default (PlotHist x Double) where
    def = defaultNormedPlotHist

plot_jpsi :: [Float] -> Renderable ()
plot_jpsi jpsis = toRenderable $ do
                    plot $ fmap histToPlot $ liftEC $ do
                        plot_hist_title .= "J/psi"
                        plot_hist_bins .= 128
                        plot_hist_values .= jpsis
                        plot_hist_norm_func .= const id

plot_jpsi <$> jpsis